In [1]:
# Test of the circuit from "Crossing a topological phase transition with a quantum computer" (https://arxiv.org/pdf/1910.05351.pdf)

In [3]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.utils import map_to_radians
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r
from qsgenerator.phase.circuits import build_ground_state_circuit, build_u1_gate, build_u_gate
from qsgenerator.states.simple_state_circuits import build_x_rotation_state, get_arcsin_x_rotation_provider


In [38]:
size = 5# size of quantum circuit excluding boundary qubits

In [39]:
g = -0.5 # G parameter from the paper 

In [40]:
H = construct_hamiltonian(size, g)
lam, V = np.linalg.eigh(H)

# ground state wavefunction
psi = V[:, 0] / np.linalg.norm(V[:, 0])

In [22]:
real, real_symbols = build_ground_state_circuit(size=size, full_circuit=True)

In [23]:
real

(0, 0): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 1): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   │                                       │
(0, 2): ─────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                   │                                       │
(0, 3): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────
                                                                                                                                                                                                                   │                                       │
(0, 4): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───

In [24]:
resolver = cirq.ParamResolver({'theta_v': get_theta_v(g), 'theta_w': get_theta_w(g), 'theta_r': get_theta_r(g)})

In [25]:
resolved = cirq.resolve_parameters(real, resolver)

In [26]:
final_state = cirq.final_state_vector(resolved)

In [27]:
final_state

array([ 0.22222227+0.j,  0.31426978+0.j,  0.22222224+0.j, -0.15713488+0.j,
       -0.1111111 +0.j, -0.15713485+0.j,  0.22222221+0.j, -0.15713486+0.j,
       -0.11111113+0.j, -0.15713489+0.j, -0.11111112+0.j,  0.07856744+0.j,
       -0.11111112+0.j, -0.15713486+0.j,  0.22222224+0.j, -0.15713488+0.j,
       -0.15713489+0.j, -0.22222227+0.j, -0.15713489+0.j,  0.11111113+0.j,
        0.07856743+0.j,  0.11111113+0.j, -0.15713486+0.j,  0.11111113+0.j,
       -0.15713486+0.j, -0.22222225+0.j, -0.15713485+0.j,  0.11111112+0.j,
       -0.15713486+0.j, -0.22222225+0.j,  0.31426978+0.j, -0.22222227+0.j],
      dtype=complex64)

In [29]:
cirq.partial_trace_of_state_vector_as_mixture(final_state, [1,2,3]) 

((0.22222226858139038,
  array([ 2.3407293e-07+0.j,  6.5817815e-01+0.j, -9.3004566e-08+0.j,
          6.5817803e-01+0.j, -2.5846004e-01+0.j, -3.0420367e-07+0.j,
         -2.5846002e-01+0.j,  5.3195316e-07+0.j], dtype=complex64)),
 (0.22222229838371277,
  array([ 1.1738812e-07+0.j,  2.5846004e-01+0.j, -8.7461977e-08+0.j,
          2.5846002e-01+0.j,  6.5817809e-01+0.j, -6.8517842e-08+0.j,
          6.5817809e-01+0.j,  5.6072867e-08+0.j], dtype=complex64)),
 (0.27777785062789917,
  array([ 5.1132226e-01-0.j, -4.5653070e-07+0.j, -2.5566110e-01+0.j,
         -4.6998338e-07+0.j,  5.8686357e-08+0.j, -3.6692965e-01+0.j,
          7.9258449e-08+0.j,  7.3385936e-01+0.j], dtype=complex64)),
 (0.27777791023254395,
  array([-7.3385942e-01+0.j, -8.9657895e-08+0.j,  3.6692962e-01+0.j,
         -6.2481838e-08+0.j,  9.9369117e-08+0.j, -2.5566107e-01+0.j,
          6.9843672e-08+0.j,  5.1132226e-01+0.j], dtype=complex64)))

In [41]:
psi

array([-0.36214298,  0.18107149,  0.18107149,  0.18107149,  0.18107149,
       -0.09053575,  0.18107149,  0.18107149,  0.18107149, -0.09053575,
       -0.09053575, -0.09053575,  0.18107149, -0.09053575,  0.18107149,
        0.18107149,  0.18107149,  0.18107149, -0.09053575,  0.18107149,
       -0.09053575, -0.09053575, -0.09053575,  0.18107149,  0.18107149,
        0.18107149, -0.09053575,  0.18107149,  0.18107149,  0.18107149,
        0.18107149, -0.36214298])

In [143]:
first_qubit_zero_mask = [1 if len(final_state)/2 > i else 0 for i in range(len(final_state)) ]
last_qubit_zero_mask = [1 if i %2 == 0 else 0 for i in range(len(final_state))]

In [148]:
first_qubit = next(q for q in real.all_qubits() if q.col == 0)
last_qubit = next(q for q in real.all_qubits() if q.col == len(real.all_qubits()) - 1)


In [149]:
realm = real + cirq.measure(first_qubit) + cirq.measure(last_qubit) 
resolvedm = cirq.resolve_parameters(realm, resolver)

In [36]:
resolved

(0, 0): ───H───@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 1): ───────X───Z───Ry(0.608π)───X────────────@───X─────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 │                                     │
(0, 2): ────────────────────────────Ry(0.696π)───X───X───Ry(0.696π)───X───Ry(0.196π)───X───X───Ry(0.196π)───X───X────────────@───X─────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                             │                                     │
(0, 3): ────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(0.696π)───X───X───Ry(0.696π)───X───Ry(0.196π)───X───X───Ry(0.196π)───X───X────────────@───X─────────────────────────────────@───X────────────────────
                                                                                                                                                                                                         │                                     │
(0, 4): ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(0.696π)───X───X───Ry(0.696π)───X───Ry(0.196π)───X───X───Ry(0.196π)───X───

In [43]:
cirq.Simulator().simulate(resolved)

measurements: (no measurements)
output vector: [ 0.22222227+0.j  0.31426978+0.j  0.22222224+0.j -0.15713488+0.j
 -0.1111111 +0.j -0.15713485+0.j  0.22222221+0.j -0.15713486+0.j
 -0.11111113+0.j -0.15713489+0.j -0.11111112+0.j  0.07856744+0.j
 -0.11111112+0.j -0.15713486+0.j  0.22222224+0.j -0.15713488+0.j
 -0.15713489+0.j -0.22222227+0.j -0.15713489+0.j  0.11111113+0.j
  0.07856743+0.j  0.11111113+0.j -0.15713486+0.j  0.11111113+0.j
 -0.15713486+0.j -0.22222225+0.j -0.15713485+0.j  0.11111112+0.j
 -0.15713486+0.j -0.22222225+0.j  0.31426978+0.j -0.22222227+0.j]

In [166]:
fs = cirq.final_state_vector(resolvedm) 
fs

array([0.       +0.j, 0.9999999+0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j,
       0.       +0.j, 0.       +0.j, 0.       +0.j, 0.       +0.j],
      dtype=complex64)

In [170]:
cirq.partial_trace_of_state_vector_as_mixture(fs, [1,2,3]) 

((0.49999991059303284,
  array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        dtype=complex64)),
 (0.49999991059303284,
  array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        dtype=complex64)))

In [169]:
psi

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [98]:
# TODO: plug in 0 instead of tracing-out
# TODO: renormalize
partials = cirq.partial_trace_of_state_vector_as_mixture(final_state, [4])
partials

((0.49999991059303284, array([1.+0.j, 0.+0.j], dtype=complex64)),
 (0.49999991059303284, array([0.+0.j, 1.+0.j], dtype=complex64)))

In [18]:
for p in partials:
    print(p[0], cirq.fidelity(p[1], psi))

0.2499997466802597 0.29109976691641815
0.2499997466802597 0.1830890225454351
0.2499997764825821 0.2089002146716102
0.2499997764825821 0.3169109914231965


In [110]:
g = 0.8

q1, q2 = cirq.GridQubit.rect(1, 2)
cirq.unitary(build_u1_gate(q1, q2, theta_r))

array([[ 0.47140452+0.j,  0.52704628+0.j,  0.47140452+0.j,
         0.52704628+0.j],
       [ 0.52704628+0.j, -0.47140452+0.j,  0.52704628+0.j,
        -0.47140452+0.j],
       [ 0.52704628+0.j,  0.47140452+0.j, -0.52704628+0.j,
        -0.47140452+0.j],
       [-0.47140452+0.j,  0.52704628+0.j,  0.47140452+0.j,
        -0.52704628+0.j]])

In [111]:
size
x_rot, x_rot_symbols = build_x_rotation_state(size=size)

In [112]:
x_resolver = cirq.ParamResolver({'r0': get_theta_v(g), 'r1': get_theta_w(g), 'r2': get_theta_r(g)})

In [113]:
x_resolved = cirq.resolve_parameters(x_rot, x_resolver)

In [109]:
g05 = cirq.final_state_vector(x_resolved)

In [114]:
g1 = cirq.final_state_vector(x_resolved)

In [115]:
g1

array([ 0.56851923+0.j        ,  0.        -0.63562375j,
        0.        -0.2542495j , -0.2842596 +0.j        ,
        0.        -0.21715502j, -0.24278669+0.j        ,
       -0.09711468+0.j        ,  0.        +0.10857751j], dtype=complex64)

In [116]:
g05

array([ 0.2542495 +0.j        ,  0.        -0.2842596j ,
        0.        -0.56851923j, -0.63562375+0.j        ,
        0.        -0.09711468j, -0.10857751+0.j        ,
       -0.21715502+0.j        ,  0.        +0.24278669j], dtype=complex64)

In [117]:
cirq.fidelity(g1, g05)

0.5555555126335463

In [55]:
circuit = cirq.Circuit()
qs = cirq.GridQubit.rect(1, 3)
s = sympy.symbols("xd")
for q in qs:
    circuit.append([cirq.rx(s).on(q)])

In [80]:
def resolved_rx(rot, c):
    resolverx = cirq.ParamResolver({"xd": map_to_radians(rot)})
    resolved = cirq.resolve_parameters(c, resolverx)
    return cirq.Simulator().simulate(resolved)

In [82]:
resolved_rx(-1, circuit)

measurements: (no measurements)
output vector: |000⟩

In [83]:
resolved_rx(-0.75, circuit)

measurements: (no measurements)
output vector: 0.943|000⟩ - 0.188j|001⟩ - 0.188j|010⟩ - 0.037|011⟩ - 0.188j|100⟩ - 0.037|101⟩ - 0.037|110⟩ + 0.007j|111⟩

In [84]:
resolved_rx(-0.5, circuit)

measurements: (no measurements)
output vector: 0.789|000⟩ - 0.327j|001⟩ - 0.327j|010⟩ - 0.135|011⟩ - 0.327j|100⟩ - 0.135|101⟩ - 0.135|110⟩ + 0.056j|111⟩

In [85]:
resolved_rx(-0.25, circuit)

measurements: (no measurements)
output vector: 0.575|000⟩ - 0.384j|001⟩ - 0.384j|010⟩ - 0.257|011⟩ - 0.384j|100⟩ - 0.257|101⟩ - 0.257|110⟩ + 0.171j|111⟩

In [86]:
resolved_rx(0, circuit)

measurements: (no measurements)
output vector: 0.354|000⟩ - 0.354j|001⟩ - 0.354j|010⟩ - 0.354|011⟩ - 0.354j|100⟩ - 0.354|101⟩ - 0.354|110⟩ + 0.354j|111⟩